# Analisi dell'importanza delle features
In questo laboratorio utilizzerai una Random Forest per valutare l’importanza relativa delle features del set di addestramento. Questa tecnica viene spesso utilizzata per eliminare le features irrilevanti prima dell’addestramento.

Utilizzerai un dataset contenente traffico benigno e vari tipi di attacchi DDoS, estratto dal dataset CIC-DDoS2019 (https://www.unb.ca/cic/datasets/ddos-2019.html).
Il traffico di rete è stato precedentemente pre-elaborato in modo che i pacchetti siano raggruppati in flussi di traffico bidirezionali utilizzando il 5-tuple (IP sorgente, IP destinazione, Porta sorgente, Porta destinazione, protocollo).
Ogni flusso è rappresentato da 21 features dell’header dei pacchetti, calcolate su un massimo di 1000 pacchetti.

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (mean IAT) | 
| 01 | packet_length (mean)| 
| 02 | IP_flags_df (sum) |
| 03 | IP_flags_mf (sum) |
| 04 | IP_flags_rb (sum) | 
| 05 | IP_frag_off (sum) |
| 06 | protocols (mean) |
| 07 | TCP_length (mean) |
| 08 | TCP_flags_ack (sum) |
| 09 | TCP_flags_cwr (sum) |
| 10 | TCP_flags_ece (sum) |
| 11 | TCP_flags_fin (sum) |
| 12 | TCP_flags_push (sum) |
| 13 | TCP_flags_res (sum) |
| 14 | TCP_flags_reset (sum) |
| 15 | TCP_flags_syn (sum) |
| 16 | TCP_flags_urg (sum) |
| 17 | TCP_window_size (mean) |
| 18 | UDP_length (mean) |
| 19 | ICMP_type (mean) |
| 20 | Packets (counter)|

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Corso di Algoritmi di Machine Learning per la rilevazione di attacchi informatici
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import numpy as np
import glob
import h5py
import sys
import copy
import argparse
from sklearn.metrics import classification_report, accuracy_score
import logging
from util_functions import *
from IPython.display import Image, display
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier

OUTPUT_FILE = "./rf_tree"
DATASET_FOLDER = "../Dataset"
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')

from matplotlib import pyplot as plt
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

SEED=1
feature_names = get_feature_names()
target_names = ['benign', 'dns',  'syn', 'udplag', 'webddos'] 
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')

In [ ]:
def show_tree(tree_clf, feature_names):
    export_graphviz(
        tree_clf,
        out_file=OUTPUT_FILE + ".dot",
        feature_names=feature_names,
        class_names=target_names,
        rounded=True,
        filled=True
    )

    # comvert the "dot" file into a png image
    os.system("dot -Tpng " + OUTPUT_FILE + ".dot -o " + OUTPUT_FILE + ".png")
    display(Image(filename=OUTPUT_FILE + ".png"))

# Classification with Random Forests
Implementa un classificatore Random Forest con 100 alberi (```estimators```) e sperimenta con gli iperparametri di regolarizzazione, come ```max_depth```, ```min_samples_split```, ```min_samples_leaf```.

In [ ]:
### Riempi gli spazi dopo i segni di uguale
rf = RandomForestClassifier(n_estimators=  , max_depth= , min_samples_split= , oob_score=True)
rf.fit(X_train,y_train)
##########################

# Feature importance
Ora visualizziamo un grafico delle caratteristiche più importanti, calcolate utilizzando la riduzione media dell’impurità di Gini.

In [ ]:
### Assign the feature importances to "fi"
fi = rf.feature_importances_
plt.barh(feature_names, fi)
plt.show()

# Inference using the RF model
Usa la Random Forest addestrata per fare previsioni sul set di test.

In [ ]:
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')

### Completa il codice tra le parentesi
y_pred = rf.predict( )

# Stampiamo i risultati della classificazione

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

# Matrice di confusione multi-classe
Ora ritorniamo alle nostre 5 classi di traffic (benigno e 4 attacchi) e analizziamo in dettaglio il risultato della classificazione della Random Forest.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report, confusion_matrix
np.set_printoptions(precision=2)

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()